# Selected Economic Characteristics: Income from the American Community Survey

**[Work in progress]**

This notebook downloads [selected economic characteristics (DP03)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-9f7418e6-ef5d-4a2d-ae16-29a5a6814849/installation-4.1.0/import


## Download selected variables

* [Selected economic characteristics for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP03 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# INCOME AND BENEFITS
             'DP03_0051E': 'totalHouseholds',
             'DP03_0052E': 'householdIncomeLessThan10000USD',
             'DP03_0052PE': 'householdIncomeLessThan10000USDPct',
             'DP03_0053E': 'householdIncome10000To14999USD',
             'DP03_0053PE': 'householdIncome10000To14999USDPct',
             'DP03_0054E': 'householdIncome15000To24999USD',
             'DP03_0054PE': 'householdIncome15000To24999USDPct',
             'DP03_0055E': 'householdIncome25000To34999USD',
             'DP03_0055PE': 'householdIncome25000To34999USDPct',
             'DP03_0056E': 'householdIncome35000To49999USD',
             'DP03_0056PE': 'householdIncome35000To49999USDPct',
             'DP03_0057E': 'householdIncome50000To74999USD',
             'DP03_0057PE': 'householdIncome50000To74999USDPct',
             'DP03_0058E': 'householdIncome75000To99999USD',
             'DP03_0058PE': 'householdIncome75000To99999USDPct',
             'DP03_0059E': 'householdIncome100000To149999USD',
             'DP03_0059PE': 'householdIncome100000To149999USDPct',
             'DP03_0060E': 'householdIncome150000To199999USD',
             'DP03_0060PE': 'householdIncome150000To199999USDPct',
             'DP03_0061E': 'householdIncomeMoreThan200000USD',
             'DP03_0061PE': 'householdIncomeMoreThan200000USDPct',
             'DP03_0062E': 'medianHouseholdIncomeUSD',
             'DP03_0063E': 'meanHouseholdIncomeUSD',
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    if 'Pct' in v:
        print('i.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('i.' + v + ' = toInteger(row.' + v + '),')

i.totalHouseholds = toInteger(row.totalHouseholds),
i.householdIncomeLessThan10000USD = toInteger(row.householdIncomeLessThan10000USD),
i.householdIncomeLessThan10000USDPct = toFloat(row.householdIncomeLessThan10000USDPct),
i.householdIncome10000To14999USD = toInteger(row.householdIncome10000To14999USD),
i.householdIncome10000To14999USDPct = toFloat(row.householdIncome10000To14999USDPct),
i.householdIncome15000To24999USD = toInteger(row.householdIncome15000To24999USD),
i.householdIncome15000To24999USDPct = toFloat(row.householdIncome15000To24999USDPct),
i.householdIncome25000To34999USD = toInteger(row.householdIncome25000To34999USD),
i.householdIncome25000To34999USDPct = toFloat(row.householdIncome25000To34999USDPct),
i.householdIncome35000To49999USD = toInteger(row.householdIncome35000To49999USD),
i.householdIncome35000To49999USDPct = toFloat(row.householdIncome35000To49999USDPct),
i.householdIncome50000To74999USD = toInteger(row.householdIncome50000To74999USD),
i.householdIncome50000

In [7]:
print(len(variables.keys()))

23


## Download county-level data using US Census API

In [8]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [9]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,DP03_0051E,DP03_0052E,DP03_0052PE,DP03_0053E,DP03_0053PE,DP03_0054E,DP03_0054PE,DP03_0055E,DP03_0055PE,DP03_0056E,DP03_0056PE,DP03_0057E,DP03_0057PE,DP03_0058E,DP03_0058PE,DP03_0059E,DP03_0059PE,DP03_0060E,DP03_0060PE,DP03_0061E,DP03_0061PE,DP03_0062E,DP03_0063E,state,county
1,18299,2766,15.1,1845,10.1,3056,16.7,2480,13.6,2153,11.8,2891,15.8,1326,7.2,1022,5.6,333,1.8,427,2.3,30834,49433,28,151
2,4563,421,9.2,410,9.0,757,16.6,628,13.8,548,12.0,714,15.6,581,12.7,240,5.3,217,4.8,47,1.0,39007,55077,28,111
3,3164,286,9.0,352,11.1,515,16.3,340,10.7,410,13.0,541,17.1,392,12.4,227,7.2,46,1.5,55,1.7,37203,49162,28,019
4,8706,585,6.7,612,7.0,1321,15.2,1248,14.3,1217,14.0,1816,20.9,827,9.5,717,8.2,270,3.1,93,1.1,40510,53717,28,057


##### Add column names

In [10]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [11]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [12]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [13]:
df.head()

,totalHouseholds,householdIncomeLessThan10000USD,householdIncomeLessThan10000USDPct,householdIncome10000To14999USD,householdIncome10000To14999USDPct,householdIncome15000To24999USD,householdIncome15000To24999USDPct,householdIncome25000To34999USD,householdIncome25000To34999USDPct,householdIncome35000To49999USD,householdIncome35000To49999USDPct,householdIncome50000To74999USD,householdIncome50000To74999USDPct,householdIncome75000To99999USD,householdIncome75000To99999USDPct,householdIncome100000To149999USD,householdIncome100000To149999USDPct,householdIncome150000To199999USD,householdIncome150000To199999USDPct,householdIncomeMoreThan200000USD,householdIncomeMoreThan200000USDPct,medianHouseholdIncomeUSD,meanHouseholdIncomeUSD,stateFips,countyFips
1,18299,2766,15.1,1845,10.1,3056,16.7,2480,13.6,2153,11.8,2891,15.8,1326,7.2,1022,5.6,333,1.8,427,2.3,30834,49433,28,151
2,4563,421,9.2,410,9.0,757,16.6,628,13.8,548,12.0,714,15.6,581,12.7,240,5.3,217,4.8,47,1.0,39007,55077,28,111
3,3164,286,9.0,352,11.1,515,16.3,340,10.7,410,13.0,541,17.1,392,12.4,227,7.2,46,1.5,55,1.7,37203,49162,28,019
4,8706,585,6.7,612,7.0,1321,15.2,1248,14.3,1217,14.0,1816,20.9,827,9.5,717,8.2,270,3.1,93,1.1,40510,53717,28,057
5,3658,377,10.3,192,5.2,531,14.5,408,11.2,515,14.1,660,18.0,350,9.6,420,11.5,92,2.5,113,3.1,43060,57919,28,015


In [14]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,totalHouseholds,householdIncomeLessThan10000USD,householdIncomeLessThan10000USDPct,householdIncome10000To14999USD,householdIncome10000To14999USDPct,householdIncome15000To24999USD,householdIncome15000To24999USDPct,householdIncome25000To34999USD,householdIncome25000To34999USDPct,householdIncome35000To49999USD,householdIncome35000To49999USDPct,householdIncome50000To74999USD,householdIncome50000To74999USDPct,householdIncome75000To99999USD,householdIncome75000To99999USDPct,householdIncome100000To149999USD,householdIncome100000To149999USDPct,householdIncome150000To199999USD,householdIncome150000To199999USDPct,householdIncomeMoreThan200000USD,householdIncomeMoreThan200000USDPct,medianHouseholdIncomeUSD,meanHouseholdIncomeUSD,stateFips,countyFips
1869,1118980,51029,4.6,39441,3.5,78317,7.0,84271,7.5,122572,11.0,184805,16.5,144979,13.0,197632,17.7,98349,8.8,117585,10.5,74855,101302,06,073


In [15]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [16]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03IncomeAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,DP03_0051E,DP03_0052E,DP03_0052PE,DP03_0053E,DP03_0053PE,DP03_0054E,DP03_0054PE,DP03_0055E,DP03_0055PE,DP03_0056E,DP03_0056PE,DP03_0057E,DP03_0057PE,DP03_0058E,DP03_0058PE,DP03_0059E,DP03_0059PE,DP03_0060E,DP03_0060PE,DP03_0061E,DP03_0061PE,DP03_0062E,DP03_0063E,zip code tabulation area
1,3811,345,9.1,304,8.0,489,12.8,475,12.5,601,15.8,732,19.2,359,9.4,386,10.1,69,1.8,51,1.3,42826,54372,43964
2,19911,1345,6.8,956,4.8,2113,10.6,2473,12.4,3311,16.6,3653,18.3,2229,11.2,2379,11.9,896,4.5,556,2.8,48647,65668,28216
3,26695,485,1.8,297,1.1,1040,3.9,1284,4.8,1968,7.4,3690,13.8,3720,13.9,5599,21.0,3542,13.3,5070,19.0,105885,137378,28277
4,9176,249,2.7,94,1.0,296,3.2,344,3.7,960,10.5,1734,18.9,1412,15.4,2116,23.1,1116,12.2,855,9.3,90282,111214,28278


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,totalHouseholds,householdIncomeLessThan10000USD,householdIncomeLessThan10000USDPct,householdIncome10000To14999USD,householdIncome10000To14999USDPct,householdIncome15000To24999USD,householdIncome15000To24999USDPct,householdIncome25000To34999USD,householdIncome25000To34999USDPct,householdIncome35000To49999USD,householdIncome35000To49999USDPct,householdIncome50000To74999USD,householdIncome50000To74999USDPct,householdIncome75000To99999USD,householdIncome75000To99999USDPct,householdIncome100000To149999USD,householdIncome100000To149999USDPct,householdIncome150000To199999USD,householdIncome150000To199999USDPct,householdIncomeMoreThan200000USD,householdIncomeMoreThan200000USDPct,medianHouseholdIncomeUSD,meanHouseholdIncomeUSD,postalCode
1,3811,345,9.1,304,8.0,489,12.8,475,12.5,601,15.8,732,19.2,359,9.4,386,10.1,69,1.8,51,1.3,42826,54372,43964
2,19911,1345,6.8,956,4.8,2113,10.6,2473,12.4,3311,16.6,3653,18.3,2229,11.2,2379,11.9,896,4.5,556,2.8,48647,65668,28216
3,26695,485,1.8,297,1.1,1040,3.9,1284,4.8,1968,7.4,3690,13.8,3720,13.9,5599,21.0,3542,13.3,5070,19.0,105885,137378,28277
4,9176,249,2.7,94,1.0,296,3.2,344,3.7,960,10.5,1734,18.9,1412,15.4,2116,23.1,1116,12.2,855,9.3,90282,111214,28278
5,13207,1030,7.8,960,7.3,1842,13.9,2027,15.3,2268,17.2,2063,15.6,1286,9.7,950,7.2,374,2.8,407,3.1,39896,59174,28303


In [21]:
# Example data
df.query("postalCode == '90210'")

,totalHouseholds,householdIncomeLessThan10000USD,householdIncomeLessThan10000USDPct,householdIncome10000To14999USD,householdIncome10000To14999USDPct,householdIncome15000To24999USD,householdIncome15000To24999USDPct,householdIncome25000To34999USD,householdIncome25000To34999USDPct,householdIncome35000To49999USD,householdIncome35000To49999USDPct,householdIncome50000To74999USD,householdIncome50000To74999USDPct,householdIncome75000To99999USD,householdIncome75000To99999USDPct,householdIncome100000To149999USD,householdIncome100000To149999USDPct,householdIncome150000To199999USD,householdIncome150000To199999USDPct,householdIncomeMoreThan200000USD,householdIncomeMoreThan200000USDPct,medianHouseholdIncomeUSD,meanHouseholdIncomeUSD,postalCode
30897,8036,509,6.3,269,3.3,366,4.6,214,2.7,258,3.2,485,6.0,745,9.3,1287,16.0,729,9.1,3174,39.5,143542,285015,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03IncomeZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [24]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [25]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [26]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [27]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [28]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,totalHouseholds,householdIncomeLessThan10000USD,householdIncomeLessThan10000USDPct,householdIncome10000To14999USD,householdIncome10000To14999USDPct,householdIncome15000To24999USD,householdIncome15000To24999USDPct,householdIncome25000To34999USD,householdIncome25000To34999USDPct,householdIncome35000To49999USD,householdIncome35000To49999USDPct,householdIncome50000To74999USD,householdIncome50000To74999USDPct,householdIncome75000To99999USD,householdIncome75000To99999USDPct,householdIncome100000To149999USD,householdIncome100000To149999USDPct,householdIncome150000To199999USD,householdIncome150000To199999USDPct,householdIncomeMoreThan200000USD,householdIncomeMoreThan200000USDPct,medianHouseholdIncomeUSD,meanHouseholdIncomeUSD,stateFips,countyFips,tract,source,aggregationLevel
56,2903,94,3.2,0,0.0,50,1.7,26,0.9,118,4.1,316,10.9,363,12.5,358,12.3,367,12.6,1211,41.7,166383,208333,06,073,06073008324,American Community Survey 5 year,Tract
57,819,137,16.7,52,6.3,46,5.6,70,8.5,26,3.2,63,7.7,162,19.8,157,19.2,70,8.5,36,4.4,77039,80888,06,073,06073008339,American Community Survey 5 year,Tract
58,2180,49,2.2,0,0.0,102,4.7,161,7.4,103,4.7,267,12.2,290,13.3,508,23.3,352,16.1,348,16.0,112368,130806,06,073,06073008347,American Community Survey 5 year,Tract
59,3200,37,1.2,51,1.6,71,2.2,87,2.7,336,10.5,466,14.6,422,13.2,839,26.2,572,17.9,319,10.0,110455,121883,06,073,06073008354,American Community Survey 5 year,Tract
60,2116,177,8.4,89,4.2,103,4.9,138,6.5,255,12.1,381,18.0,252,11.9,428,20.2,165,7.8,128,6.0,63625,90850,06,073,06073008505,American Community Survey 5 year,Tract


### Save data

In [29]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03IncomeTract.csv", index=False)

In [30]:
df.shape

(73056, 28)